In [1]:
import sqlite3

In [2]:
import pandas as pd

In [9]:
# Read a small chunk of the CSV file to inspect column names and data types
small_chunk = pd.read_csv('TMDbTrendingMovies.csv', nrows=10)
print(small_chunk.dtypes)


id                     int64
title                 object
original_language     object
original_title        object
overview              object
genre_ids             object
popularity           float64
release_date          object
vote_average         float64
vote_count             int64
dtype: object


In [12]:
import pandas as pd
import sqlite3

# Step 1: Inspect the CSV file
# Read a small chunk of the CSV file to inspect column names and data types
small_chunk = pd.read_csv('TMDbTrendingMovies.csv', nrows=10)
print(small_chunk.dtypes)
print(small_chunk.head())

# Step 2: Dynamically create the CREATE TABLE query based on inspection
def create_table_query_from_dtypes(df, table_name):
    dtypes_mapping = {
        'int64': 'INTEGER',
        'float64': 'REAL',
        'object': 'TEXT',
        'bool': 'INTEGER',
        'datetime64[ns]': 'TEXT'
    }

    columns = []
    for column, dtype in df.dtypes.items():
        sql_dtype = dtypes_mapping.get(str(dtype), 'TEXT')
        columns.append(f'"{column}" {sql_dtype}')
    
    columns_str = ",\n    ".join(columns)
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns_str}
    )
    '''
    return create_table_query

table_name = 'tmdb'
create_table_query = create_table_query_from_dtypes(small_chunk, table_name)
print(create_table_query)  # Print the query to verify

# Step 3: Create the table and insert data in chunks
# Create an SQLite database connection
conn = sqlite3.connect('tmdb.db')
cursor = conn.cursor()

# Execute the CREATE TABLE query
cursor.execute(create_table_query)
conn.commit()

# Define the chunk size
chunk_size = 10000

# Read the CSV file in chunks and insert into the database
for chunk in pd.read_csv('TMDbTrendingMovies.csv', chunksize=chunk_size):
    chunk.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection
conn.close()


id                     int64
title                 object
original_language     object
original_title        object
overview              object
genre_ids             object
popularity           float64
release_date          object
vote_average         float64
vote_count             int64
dtype: object
       id                     title original_language  \
0  766507                      Prey                en   
1  361743         Top Gun: Maverick                en   
2  755566                 Day Shift                en   
3  438148  Minions: The Rise of Gru                en   
4  614934                     Elvis                en   

             original_title  \
0                      Prey   
1         Top Gun: Maverick   
2                 Day Shift   
3  Minions: The Rise of Gru   
4                     Elvis   

                                            overview         genre_ids  \
0  When danger threatens her camp, the fierce and...     [28, 878, 53]   
1  After more than